In [1]:
#import all external packages
import re, os, sys, time
import pandas as pd

#import all internal code libraries
import tools, bio_tools, compute_features
import parse_gwas, get_dbSNP

#set organism information for host and virus
host = '9606'
host_organism = 'homo sapiens'
virus = '2697049'
virus_organism = "Severe acute respiratory coronavirus 2"

#set locations of necessary files for cross-referencing
directory = "./"
biogrid_file = "BIOGRID-ALL-4.4.206.tab3.txt"
go_file = "coagulation_proteins.tsv"
gwas_file = "COVID19_HGI_A2_ALL_leave_23andme_20210607.b37.txt"
gwas_prefix = "A2"

In [2]:
#parse biogrid file and filter for only interactions involving both the host and virus organisms
biogrid = pd.read_csv(os.path.join(directory, biogrid_file), sep="\t", header=0, low_memory=False).T.to_dict()
biogrid = {k:v for k,v in biogrid.items() if str(v["Organism ID Interactor A"]) in [host, virus] and str(v["Organism ID Interactor B"]) in [host, virus]}
genes = []
for k,v in biogrid.items():
    if str(v["Organism ID Interactor A"]) == host and str(v["Organism ID Interactor B"]) == virus:
        genes.append(v["Official Symbol Interactor A"])
    elif str(v["Organism ID Interactor B"]) == host and str(v["Organism ID Interactor A"]) == virus:
        genes.append(v["Official Symbol Interactor B"])
genes = {gene:None for gene in genes}

In [3]:
#parse gene ontology output (used for coagulation here), and gather all related gene names
#cross-reference against genes from gene ontology
coagulation = pd.read_csv(os.path.join(directory, go_file), sep="\t", header=None)
coagulation = set(coagulation[0]) #this index may need to change based on the location of the gene name
coagulation = [x for x in coagulation if x in genes]
genes = {k:v for k,v in genes.items() if k in coagulation}

In [ ]:
#for each gene of interest, collect accession IDs (RefSeq)
for gene in list(genes):
    try:
        accids = bio_tools.get_accids(gene, organism=host_organism)["mRNA"]
        accids = [x[0] for x in accids]
        genes[gene] = bio_tools.accid_opt(accids)
    except Exception as e:
        del genes[gene]
genes

IncompleteRead(397503 bytes read)
local variable 'genomic' referenced before assignment
IncompleteRead(852159 bytes read)
local variable 'genomic' referenced before assignment


In [ ]:
#for each gene of interest, query RefSeq for genomic, transcript, and ORF sequences
#then, query dbSNP for all synonymous and missense variants in gene and score them
missense = {}
synonymous = {}
init_time = time.time()
if not os.path.isdir("./seqdir/"):
    os.mkdir("./seqdir/")
for i, (genename, nm) in enumerate(genes.items()):
    if any([True if k.startswith(genename) else False for k in list(missense.keys()) + list(synonymous.keys())]):
        continue
    if os.path.isfile("./seqdir/" + genename + "_" + nm + ".fasta"):
        seqs = tools.read_fasta("./seqdir/" + genename + "_" + nm + ".fasta")
    else:
        try:
            seqs = bio_tools.get_all_seqs(genename, nm, write=True, outdir="./seqdir/")["seqs"]
        except Exception as e:
            print(genename + "\t" + str(e))
            continue
    try:
        tmpd = get_dbSNP.get_syn(genename, seqs["ORF"], nm)
        for k,v in tmpd.items():
            k = k[:-1] + ">" + k[-1]
            synonymous[genename + ":c." + k] = v
    except Exception as e:
        print(genename + "\t" + str(e))
    tools.pickledump(os.path.join(directory, synonymous, "coag_syn.pkl"))
    try:
        tmpd = get_dbSNP.get_nonsyn(genename, seqs["ORF"], nm)
        for k,v in tmpd.items():
            k = k[:-1] + ">" + k[-1]
            missense[genename + ":c." + k] = v
    except Exception as e:
        print(genename + "\t" + str(e))
    tools.pickledump(os.path.join(directory, missense, "coag_missense.pkl"))
    tools.update_time(i, len(genes), init_time)
missense = pd.DataFrame(missense).T
synonymous = pd.DataFrame(synonymous).T
missense.to_csv(os.path.join(directory, gwas_prefix + "_missense.tsv"), sep="\t")
synonymous.to_csv(os.path.join(directory, gwas_prefix + "_synonymous.tsv"), sep="\t")

In [ ]:
#filter the GWAS data for only regions including the genes of interest
#additionally score these variants for splicing impact (many of these are intronic or UTR)
gwas_data = parse_gwas.gwas_pipeline(gwas_file, list(genes.keys()), prefix=gwas_prefix, index_col="SNP", p_col="all_inv_var_meta_p", chromosome="#CHR", position="POS", ref="REF", alt="ALT", plim=0.05, translation=gwas_prefix + ".trans")
pd.DataFrame(gwas_data).T.to_csv(gwas_prefix + "_sub.tsv", sep="\t")

In [ ]:
pd.DataFrame(gwas_data).T

In [ ]:
#count number of extreme values (splicing scores, conservation, mRNA MFE, MAF, etc) for each variant, in terms of percentile rank relative to the set of variants
for name, df in {"missense":missense, "synonymous":synonymous}.items():
    def count_ext(row):
        return(len([col for col in row if tools.is_numeric(row[col]) and tools.is_numeric(tools.pctile(df[col], row[col])) and (tools.pctile(df[col], row[col]) <= 0.05 or tools.pctile(df[col], row[col]) >= 0.95)]))
    df["Extreme values"] = df.apply(lambda x: count_ext({col:x[col] for col in df.columns}), axis=1)

In [ ]:
#sort synonymous and missense variants by the number of extreme values and output them
synonymous.sort_values("Extreme values")
missense.sort_values("Extreme values")
synonymous.to_csv(gwas_prefix + "_synonymous.tsv", sep="\t")
missense.to_csv(gwas_prefix + "_missense.tsv", sep="\t")

In [36]:
#create list of host interactors for each viral gene/protein
viral_genes = []
for k,v in biogrid.items():
    if str(v["Organism ID Interactor A"]) == virus:
        viral_genes.append(str(v["Official Symbol Interactor A"]))
    elif str(v["Organism ID Interactor B"]) == virus:
        viral_genes.append(str(v["Official Symbol Interactor B"]))
viral_genes = {k:[] for k in viral_genes}
for k,v in biogrid.items():
    if str(v["Organism ID Interactor B"]) == host and str(v["Organism ID Interactor A"]) == virus and str(v["Official Symbol Interactor B"]) in genes:
        viral_genes[str(v["Official Symbol Interactor A"])].append(str(v["Official Symbol Interactor B"]))
    if str(v["Organism ID Interactor A"]) == host and str(v["Organism ID Interactor B"]) == virus and str(v["Official Symbol Interactor A"]) in genes:
        viral_genes[str(v["Official Symbol Interactor B"])].append(str(v["Official Symbol Interactor A"]))
rev_viral = {vi:[k for k in viral_genes if vi in viral_genes[k]] for v in viral_genes.values() for vi in v}

In [37]:
#create list of host interactors for each host protein of interest (useful for homology analysis)
interactor_genes = {gene:[] for gene in genes}
for k,v in biogrid.items():
    if str(v["Organism ID Interactor B"]) == host and str(v["Organism ID Interactor A"]) == host and str(v["Official Symbol Interactor B"]) in genes:
        interactor_genes[str(v["Official Symbol Interactor B"])] += [str(v["Official Symbol Interactor A"])]
    if str(v["Organism ID Interactor B"]) == host and str(v["Organism ID Interactor A"]) == host and str(v["Official Symbol Interactor A"]) in genes:
        interactor_genes[str(v["Official Symbol Interactor A"])] += [str(v["Official Symbol Interactor B"])]
rev_interactor = {vi:[k for k in interactor_genes if vi in interactor_genes[k]] for v in interactor_genes.values() for vi in v}

In [44]:
#read Dali output for each viral gene, and compare against list of host interactors for the host protein that interacts with that viral protein
#this step finds host proteins that are structurally homologous to the viral proteins and also bind to the host protein that binds to the viral protein
homologues = {}
for k,v in viral_genes.items():
    try:
        df = pd.read_csv(os.path.join(directory, k + "_dali.tsv"), sep="\t", index_col=0)
        homologues[k] = get_dbSNP.get_genename_from_pdb([[i, row["chain"]] for i,row in df.iterrows()], taxid=host)
        interactors = [yi for xi in viral_genes[k] for yi in interactor_genes[xi] if xi in interactor_genes]
        homologues[k] = {k2:v2 for k2,v2 in homologues[k].items() if any([True if vi in interactors else False for vi in v]) and len(v2) > 0}
    except Exception as e:
        print(k + "\t" + str(e))
homologues

E	[Errno 2] No such file or directory: '/media/temp/E_dali.tsv'
M	[Errno 2] No such file or directory: '/media/temp/M_dali.tsv'
S	[Errno 2] No such file or directory: '/media/temp/S_dali.tsv'
nsp1	[Errno 2] No such file or directory: '/media/temp/nsp1_dali.tsv'
nsp10	[Errno 2] No such file or directory: '/media/temp/nsp10_dali.tsv'
nsp11	[Errno 2] No such file or directory: '/media/temp/nsp11_dali.tsv'
nsp12	[Errno 2] No such file or directory: '/media/temp/nsp12_dali.tsv'
nsp13	[Errno 2] No such file or directory: '/media/temp/nsp13_dali.tsv'
nsp14	[Errno 2] No such file or directory: '/media/temp/nsp14_dali.tsv'
nsp15	[Errno 2] No such file or directory: '/media/temp/nsp15_dali.tsv'
nsp2	[Errno 2] No such file or directory: '/media/temp/nsp2_dali.tsv'
nsp4	[Errno 2] No such file or directory: '/media/temp/nsp4_dali.tsv'
nsp5	[Errno 2] No such file or directory: '/media/temp/nsp5_dali.tsv'
nsp6	[Errno 2] No such file or directory: '/media/temp/nsp6_dali.tsv'
nsp7	[Errno 2] No such fil

{'N': {'7abt_A': ['CYPA', 'PPIA']}}